# Introduction

**Table of contents**<a id='toc0_'></a>    
- 1. [VS Code](#toc1_)    
- 2. [Jupyter notebooks](#toc2_)    
- 3. [Solving the consumer problem with a numerical optimizer](#toc3_)    
- 4. [Solving the consumer problem with a brute force grid search](#toc4_)    
- 5. [Using modules](#toc5_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

**Notes:** 

1. The course content you have *downloaded* with `Git` is continously updated
2. My environment might look slightly different from yours
2. Later you will also *upload* files with `Git`

## 1. <a id='toc1_'></a>[VS Code](#toc0_)

1. An **integrated development environment (IDE)** for many languages
2. **This window:** *Editor*
3. **Side-bar on left:** *Explorer*
4. `.ipynb` are **Jupyter notebooks** with text and code
5. `.py` are files with **Python code**

**Change theme?** Try `Ctrl+Shift+P` and search for "theme".

## 2. <a id='toc2_'></a>[Jupyter notebooks](#toc0_)

This is a `text cell`, or more precisely a `markdown cell`.

* Pres <kbd>Enter</kbd> to go to *edit mode*
* Pres <kbd>Esc</kbd> to go to *command mode*
* Pres <kbd>Ctrl+Enter</kbd> to *run* the cell
* Pres <kbd>Shift+Enter</kbd> to *run* the cell + advance

We can make lists:

1. **First** item
2. *Second* item
3. ~~Third~~ item

We can also do LaTeX math, e.g. $\alpha^2$ or

$$
X = \int_0^{\infty} \frac{x}{x+1} dx
$$

**Markdown:** Guide on [markdownguide.org/basic-syntax/](https://www.markdownguide.org/basic-syntax/)

`Code cells`:

In [1]:
# this is (a comment in) a code cell
# let us do some calculations
a = 20
b = 30
c = a+b

# let us print the results (shown below the cell)
print(c)

50


We can now write some more text, and continue with our calculations.

In [2]:
d = c*2
print(d)
print(c*3)

100
150


**Task:** Write a new cell with `e = d**2` and run it.

>**Useful shortcuts**: See this [guide](https://sites.google.com/view/numeconcph-introprog/guides/vscode).
>1. Press `A` to *create* new code cell *above*
>1. Press `B` to *create* new code cell *below* 
>2. Press `M` to change to `markdown cell`
>3. Press `Y` to change to `code cell`
>4. Press `D+D` to *delete* cell
><br></br>

## 3. <a id='toc3_'></a>[Solving the consumer problem with a numerical optimizer](#toc0_)

Consider the following consumer problem:

$$
\begin{aligned}
V(p_{1},p_{2},m) & = \max_{x_{1},x_{2}} x_{1}^{\alpha}x_{2}^{1-\alpha}\\
 & \text{s.t.}\\
p_{1}x_{1}+p_{2}x_{2} & \leq m \\
x_{1},x_{2} & \geq 0 \\
p_{1},p_{2},m & > 0
\end{aligned}
$$

We can solve this problem _numerically_ in a few lines of code.

1. Choose some **parameters**:

In [3]:
alpha = 0.5
m = 10
p1 = 1
p2 = 2

**Note:** Try to click on *Variables* above to see the value of all variables.

2. The **consumer objective** is:

In [4]:
def utility(x1,x2,alpha):
    return x1**alpha * x2**(1-alpha)

# def function_name(inputs):
#    return outputs

def value_of_choice(x1,alpha,m,p1,p2):
    
    # a. all income not spent on the first good
    #    is spent on the second
    x2 = (m-p1*x1)/p2 
    
    # b. the resulting utility is
    u = utility(x1,x2,alpha)
    
    return u

3. We can now use a function from the `scipy` module to **solve the consumer problem**.

In [5]:
# a. load external module from scipy
from scipy import optimize

# b. make value-of-choice as a funciton of only x1 
obj = lambda x1: -value_of_choice(x1,alpha,m,p1,p2)

# output = lambda input: expression(input)

# c. call numerical minimizer
solution = optimize.minimize_scalar(obj,bounds=(0,m/p1))

# d. print result
u = -solution.fun
x1 = solution.x
x2 = (m-x1*p1)/p2
print(x1,x2,u)

5.000000000088435 2.4999999999557825 3.5355339059327378


**Task**: Solve the consumer problem with the CES utility funciton.

$$
u(x_1,x_2) = (\alpha x_1^{-\beta} + (1-\alpha) x_2^{-\beta})^{-1/\beta}
$$

In [6]:
# try to write your own code

In [7]:
# solution

# a. choose parameters
alpha = 0.5
beta = 0.000001
m = 10
p1 = 1
p2 = 2

# b. value-of-choice
def utility_ces(x1,x2,alpha,beta):
    if x1 > 0 and x2 > 0:
        return (alpha*x1**(-beta)+(1-alpha)*x2**(-beta))**(-1/beta) 
    else:
        return 0

def value_of_choice_ces(x1,alpha,beta,m,p1,p2):
    x2 = (m-p1*x1)/p2
    u = utility_ces(x1,x2,alpha,beta)
    return u

# c. objective
obj = lambda x1: -value_of_choice_ces(x1,alpha,beta,m,p1,p2)

# d. solve
solution = optimize.minimize_scalar(obj,bounds=(0,m/p1))

# e. result
u_ces = -solution.fun
x1_ces = solution.x
x2_ces = (m-x1_ces*p1)/p2
print(x1_ces,x2_ces,u_ces)

4.99999685598185 2.500001572009075 3.535533693620822


## 4. <a id='toc4_'></a>[Solving the consumer problem with a brute force grid search](#toc0_)

Create vectors of potential choices:

In [8]:
import numpy as np
x1_vec = np.linspace(0,10,100) # try to hover over "linspace"
x2_vec = np.linspace(0,10,100)

Use trial-and-error:

In [9]:
x1_best = np.nan # nan = Not-a-Number
x2_best = np.nan 
u_best = -np.inf # the worst it can get

for x1 in x1_vec:
    for x2 in x2_vec:
        
        if p1*x1 + p2*x2 <= m: # expenditure <= income

            u_now = utility_ces(x1,x2,alpha,beta)
        
            if u_now > u_best: # better?
                u_best = u_now # save best utility yet seen
                x1_best = x1 # ... and choice
                x2_best = x2

print(x1_best,x2_best,u_best)

4.94949494949495 2.525252525252525 3.5353533352032596


**Problem:** We get a lower utility than with the numerical optimizer...

In [10]:
print(u_best-u_ces)

-0.0001803584175625872


**Task:** Can you improve the brute force solution?

## 5. <a id='toc5_'></a>[Using modules](#toc0_)

A **module** is a **.py**-file with functions you import and can then call in the notebook.

Try to open **mymodule.py** and have a look.

In [11]:
import mymodule

In [12]:
x = 5
y = mymodule.myfunction(x) # try pressing F12 with cursor at "myfunction"
print(y)

25


**Task**: Create `mynewmodule.py` with a function you choose and call it from this notebook.